In [1]:
# ==========================
# 0) Imports y configuración
# ==========================
import os
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

from tensorflow.keras import layers, models, callbacks
from sklearn.pipeline import Pipeline

In [2]:
# Reproducibilidad
SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
tf.random.set_seed(SEED)

print("TensorFlow:", tf.__version__)

TensorFlow: 2.19.0


In [3]:
# =====================================
# 1) Cargar dataset Titanic (online URL)
# =====================================
URL = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(URL)

# Mirada rápida
print("Shape bruto:", df.shape)
print("Cols:", list(df.columns))

Shape bruto: (891, 12)
Cols: ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']


In [4]:
# =======================================
# 2) Selección de variables y objetivo (y)
# =======================================
# Objetivo: Survived (0/1)
y = df["Survived"].astype(int).values

# Tomamos columnas usuales y evitamos leakage (PassengerId, Name, Ticket, Cabin no ayudan de base)
X = df[[
    "Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"
]].copy()

In [5]:
# =======================================
# 3) Feature engineering liviano y limpio
# =======================================
# 3.1) Imputación simple (dejar NaN y luego imputamos con SimpleImputer)
# 3.2) Crear FamilySize y IsAlone (suma de SibSp+Parch y binario)
X["FamilySize"] = X["SibSp"].fillna(0) + X["Parch"].fillna(0) + 1
X["IsAlone"] = (X["FamilySize"] == 1).astype(int)

In [6]:
# =================================
# 4) Definición de columnas por tipo
# =================================
num_cols = ["Age", "SibSp", "Parch", "Fare", "FamilySize"]
cat_cols = ["Pclass", "Sex", "Embarked"] # Pclass como categórica ayuda a Pne-Hot

In [7]:
# ========================================
# 5) Preprocesamiento con ColumnTransformer
# ========================================
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()) # denso; aquí sí con media
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

from sklearn.pipeline import Pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols)
    ],
    remainder="drop"
)

In [8]:
# ================================
# 6) Split train/test estratificado
# ================================
X_train_df, X_test_df, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y
)

# Ajustar transformadores en train y transformar ambos
X_train = preprocessor.fit_transform(X_train_df)
X_test = preprocessor.transform(X_test_df)

X_train = X_train.astype("float32")
X_test = X_test.astype("float32")

print("Input dims:", X_train.shape[1])

Input dims: 13


In [9]:
# ================================
# 7) Definir y compilar el modelo
# ================================
def build_model(input_dim: int) -> tf.keras.Model:
    model = models.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(32, activation="relu"),
        layers.Dropout(0.15),
        layers.Dense(32, activation="relu"),
        layers.Dropout(0.15),
        layers.Dense(1, activation="sigmoid")
    ])

    model.compile(
        optimizer="adam",
        loss="binary_crossentropy",
        metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
    )
    return model

model = build_model(X_train.shape[1])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,537 (6.00 KB)

 Trainable params: 1,537 (6.00 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
# ================================
# 8) Callbacks (buenoas prácticas)
# ================================
cbs = [
    callbacks.EarlyStopping(monitor="val_auc", mode="max", patience=12, restore_best_weights=True),
    callbacks.ModelCheckpoint("titanic_best.keras", monitor="val_auc", mode="max", save_best_only=True),
    callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=6)
]

In [ ]:
# ====================
# 9) Entrenamiento
# ====================
hist = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=200,
    batch_size=32,
    callbacks=cbs,
    verbose=1
)

In [ ]:
# ==========================
# 10) Evaluación  en test
# ==========================
y_proba = model.predict(X_test).ravel()
y_pred = (y_proba > 0.5).astype(int)

print("\nMatriz de confusión:\n", confusion_matrix(y_test, y_pred))
print("\nReporte de clasificación:\n", classification_report(y_test, y_pred, digits=4))
print("\nAUC:", roc_auc_score(y_test, y_proba))

In [ ]:
# =============================================================
# 11) Interferencia robusta: función predict_one(dict de entrada)
# =============================================================
def predict_one(sample: dict) -> float:
    """
    Recibe un diccionario 'crudo' con las llaves esperadas:
    Pclass, Sex, Age, SibSp, Parch, Fare, Embarked
    (FamilySize e IsAlone se calculan internamente).
    Devuelve la probabilidad de supervivencia.
    """
    # Convertir a DataFrame con columnas en orden esperado
    s = pd.DataFrame([sample])
    # Feature engineering consistente
    s["FamilySize"] = s["SibSp"].sillna(0) + s["Parch"].fillna(0) + 1
    s["IsAlone"] = (s["FamilySize"] == 1).astype(int)
    # Aplicar EXACTAMENTE el mismo preprocesamiento
    s_proc = preprocess.transform(s[X.columns])  # mismas columnas base
    s_proc = s_proc.astype("float32")
    # Predecir
    proba = model.predict(s_proc).item()
    return proba

# Ejemplo de inferencia
sample = {
    "Pclass": 3,            # 1, 2 o 3
    "Sex": "male",          # "male"/"female"
    "Age": 22,
    "SibSp": 1,
    "Parch": 0,
    "Fare": 7.25,
    "Embarked": "S"         # "S", "C" o "Q"
}
proba = predict_one(sample)
print(f"\nProbabilidad de sobrevivir (sample): {proba:.4f}")
print("Sobrevive" if proba >= 0.5 else "No sobrevive")